# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nhl_correlation():
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # Keeping only columns of interest
    nhl_2018 = nhl_df[nhl_df.year == 2018]
    nhl_2018 = nhl_2018[['team', 'W', 'L', 'year']]
    
    # Cleaning teams' names in NHL DataFrame
    def clean_team(item):
      if item[-1] == '*':
        item= item[:-1]
        return item
      return item
    
    # Separating columns of interest in cities DataFrame
    cities.rename(columns={'Population (2016 est.)[8]': 'Population Estimate 2016'}, inplace=True)
    cities = cities[['Metropolitan area', 'Population Estimate 2016', 'NHL']]
    nhl_2018['team'] = nhl_2018.team.apply(clean_team)
    
    # Cleaning teams' names in cities DataFrame
    def clean_city(item):
      if '[' in item:
        return re.findall('([\w\s]*)\[', item)[0]
      return item
    cities['NHL'] = cities.NHL.apply(clean_city)
    
    # Removing divisions from NHL DataFrame
    nhl_2018 = nhl_2018[~nhl_2018.W.str.contains('Division')]
    
    # Separate team's names from cities' names
    nhl_2018['team'] = nhl_2018.team.apply(lambda y: y.rsplit(None, 1)[-1])
    nhl_2018.loc[3, 'team'] = 'Maple Leafs'
    nhl_2018.loc[5, 'team'] = 'Red Wings'
    nhl_2018.loc[13, 'team'] = 'Blue Jackets'
    nhl_2018.loc[27, 'team'] = 'Golden Knights'
    
    # Replacing names of teams from the same city
    nhl_2018.loc[14, 'team'] = 'Rangers Islanders Devils'
    nhl_2018.loc[16, 'team'] = 'Rangers Islanders Devils'
    nhl_2018.loc[17, 'team'] = 'Rangers Islanders Devils'
    nhl_2018.loc[28, 'team'] = 'Kings Ducks'
    nhl_2018.loc[30, 'team'] = 'Kings Ducks'
    
    # Asserting datatypes
    nhl_2018.W = nhl_2018.W.astype(float)
    nhl_2018.L = nhl_2018.L.astype(float)

    # Grouping by teams with same name and getting ther average W and L values
    nhl_2018 = nhl_2018.groupby(by='team', )['W', 'L'].mean()
    nhl_2018.reset_index(inplace=True)
    
    # Merging DataFrames
    cities.rename(columns={'NHL': 'team'}, inplace=True)
    merged = pd.merge(nhl_2018, cities, how='inner', on='team')
    
    # Creating Win/Loss ratio column
    merged['Win/Loss Ratio'] = merged.W / (merged.W + merged.L)

    # Passing in the numeric values of population_by_region and win_loss_by_region
    population_by_region = pd.to_numeric(merged['Population Estimate 2016'], errors='coerce')
    win_loss_by_region = pd.to_numeric(merged['Win/Loss Ratio'], errors='coerce')

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    answer = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return answer

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nba_correlation():
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Filtering data
    nba_2018 = nba_df[nba_df.year == 2018]
    
    # Cleaning teams' names in NBA DataFrame
    def clean_team(item):
      if "(" in item:
        item = re.findall('^(.*?)\s\(\d+\)$', item)[0]
        if item[-1] == '*':
          item = item[:-1]
          return item
      return item
    nba_2018['team'] = nba_2018.team.apply(clean_team)
    
    # Cleaning NBA teams names in cities DataFrame
    cities['NBA'] = cities.NBA.apply(lambda x: re.findall('([\w\s]*)\[', x)[0] if '[' in x else x)
    
    # Keeping only columns of interest
    cities.rename(columns={'Population (2016 est.)[8]': 'Population Estimate 2016', 'NBA': 'team'}, inplace=True)
    cities = cities[['Metropolitan area', 'Population Estimate 2016', 'team']]
    nba_2018 = nba_2018[['team', 'W', 'L']]
    
    # Separating team's names from cities' names and standardizing names
    nba_2018['team'] = nba_2018.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nba_2018.loc[17, 'team'] = 'Trail Blazers'
    nba_2018.loc[10, 'team'] = 'Knicks Nets'
    nba_2018.loc[11, 'team'] = 'Knicks Nets'
    nba_2018.loc[24, 'team'] = 'Lakers Clippers'
    nba_2018.loc[25, 'team'] = 'Lakers Clippers'
    
    # Asserting datatypes
    nba_2018.W = nba_2018.W.astype(float)
    nba_2018.L = nba_2018.L.astype(float)
    
    # Grouping rows with same name
    nba_2018 = nba_2018.groupby(by='team')['W', 'L'].mean()
    nba_2018.reset_index()
    
    # Merging DataFrames
    merged = pd.merge(nba_2018, cities, how='inner', on='team')
    merged.W = merged.W.astype(float)
    merged.L = merged.L.astype(float)
    
    # Creating Win/Loss Ratio column
    merged['Win/Loss Ratio'] = merged.W/(merged.W + merged.L)

    # Passing in values
    population_by_region = pd.to_numeric(merged['Population Estimate 2016'], errors='coerce')
    win_loss_by_region = pd.to_numeric(merged['Win/Loss Ratio'], errors='coerce')

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    answer = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    answer
    
    return answer

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def mlb_correlation():
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Getting 2018 data
    mlb_2018 = mlb_df[mlb_df.year == 2018]
    
    # Cleaning MLB teams names in cities DataFrame
    cities['MLB'] = cities.MLB.apply(lambda x: re.findall('([\w\s]*)\[', x)[0] if '[' in x else x)

    # Keeping only columns of interest
    mlb_2018 = mlb_2018[['team', 'W', 'L']]
    cities.rename(columns={'Population (2016 est.)[8]': 'Population Estimate 2016', 'MLB': 'team'}, inplace=True)
    cities = cities[['Metropolitan area', 'Population Estimate 2016', 'team']]
    
    # Separating team's names from cities' names and standardizing names
    mlb_2018['team'] = mlb_2018.team.apply(lambda x: x.rsplit(None, 1)[-1])
    mlb_2018.loc[13, 'team'] = 'Dodgers Angels'
    mlb_2018.loc[25, 'team'] = 'Dodgers Angels'
    mlb_2018.loc[1, 'team'] = 'Yankees Mets'
    mlb_2018.loc[18, 'team'] = 'Yankees Mets'
    mlb_2018.loc[28, 'team'] = 'Giants Athletics'
    mlb_2018.loc[11, 'team'] = 'Giants Athletics'
    mlb_2018.loc[8, 'team'] = 'Cubs White Sox'
    mlb_2018.loc[21, 'team'] = 'Cubs White Sox'
    mlb_2018.loc[0, 'team'] = 'Red Sox'
    mlb_2018.loc[3, 'team'] = 'Blue Jays'
    
    # Asserting datatype
    mlb_2018.W = mlb_2018.W.astype(float)
    mlb_2018.L = mlb_2018.L.astype(float)

    # Grouping by teams from same city
    mlb_2018 = mlb_2018.groupby(by='team')['W', 'L'].mean()
    mlb_2018.reset_index()
    
    # Merging DataFrames
    merged = pd.merge(mlb_2018, cities, how='inner', on='team')
    merged.W = merged.W.astype(float)
    merged.L = merged.L.astype(float)

    # Creating Win/Loss Ratio column
    merged['Win/Loss Ratio'] = merged.W/(merged.W + merged.L)

    # Passing in values
    population_by_region = pd.to_numeric(merged['Population Estimate 2016'], errors='coerce')
    win_loss_by_region = pd.to_numeric(merged['Win/Loss Ratio'], errors='coerce')
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    answer = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return answer

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nfl_correlation():
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Gathering and cleaning data of interest
    nfl_2018 = nfl_df[nfl_df.year == 2018]
    nfl_2018 = nfl_2018[['team', 'W', 'L']]
    nfl_2018['team'] = nfl_2018.team.apply(lambda x: re.findall('([\w\s]*)\*', x)[0] if '*' in x else x)
    nfl_2018['team'] = nfl_2018.team.apply(lambda x: re.findall('([\w\s]*)\+', x)[0] if '+' in x else x)
    nfl_2018 = nfl_2018[~nfl_2018.team.str.contains('AFC')]
    nfl_2018 = nfl_2018[~nfl_2018.team.str.contains('NFC')]
    
    # Separating teams and cities and standardizing names
    nfl_2018['team'] = nfl_2018.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nfl_2018.loc[38, 'team'] = '49ers Raiders'
    nfl_2018.loc[19, 'team'] = '49ers Raiders'
    nfl_2018.loc[17, 'team'] = 'Rams Chargers'
    nfl_2018.loc[36, 'team'] = 'Rams Chargers'
    nfl_2018.loc[4, 'team'] = 'Giants Jets'
    nfl_2018.loc[24, 'team'] = 'Giants Jets'
    
    # Keeping only columns of interest
    cities.rename(columns={'Population (2016 est.)[8]': 'Population Estimate 2016', 'NFL': 'team'}, inplace=True)
    cities = cities[['Metropolitan area', 'Population Estimate 2016', 'team']]
    cities['team'] = cities.team.apply(lambda x: re.findall('([\w\s]*)\[', x)[0] if '[' in x else x)
    
    # Asserting datatypes
    nfl_2018.W = nfl_2018.W.astype(float)
    nfl_2018.L = nfl_2018.L.astype(float)

    # Grouping teams from the same city
    nfl_2018 = nfl_2018.groupby(by='team')['W', 'L'].mean()
    nfl_2018.reset_index(inplace=True)

    # Merging DataFrames
    merged = pd.merge(nfl_2018, cities, how='inner', on='team')
    merged.W = merged.W.astype(float)
    merged.L = merged.L.astype(float)

    # Creating Win/Loss Ratio column
    merged['Win/Loss Ratio'] = merged.W/(merged.W + merged.L)

    # Passing in values
    population_by_region = pd.to_numeric(merged['Population Estimate 2016'], errors='coerce')
    win_loss_by_region = pd.to_numeric(merged['Win/Loss Ratio'], errors='coerce')

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    answer = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return answer

nfl_correlation()

0.00492211214934943

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values